In [3]:
import requests
from bs4 import BeautifulSoup
import csv
import xml.etree.ElementTree as ET
import pandas as pd

from requests.auth import HTTPBasicAuth

In [4]:
consumer_key = 'Lq3V1hAxII2Fv8YSdcrhfA538HzhsfzhelYqSUgzGU0A8AEU'
consumer_secret_key = 'jHJjRSzVQBjCizDYmFg4cqWza2R8TOCcLaecRbXrJQcseLu8Lr0AqYXL1ldkAx8A'

# EPO OAuth2 token URL
token_url = "https://ops.epo.org/3.2/auth/accesstoken"

# Request access token
response = requests.post(
    token_url,
    auth=HTTPBasicAuth(consumer_key, consumer_secret_key),
    data={"grant_type": "client_credentials"},
)

# Extract token
if response.status_code == 200:
    token = response.json()["access_token"]
    print("Access Token:", token)
else:
    print("Failed to get token:", response.text)

Access Token: UgUQAmW9AximGG4FOYplNnEn7QKA


In [15]:
# Define search URL
search_url = "https://ops.epo.org/rest-services/published-data/search"

# Headers with authentication
headers = {
    "Authorization": f"Bearer {token}",
    "Accept": "application/xml",
}

# Parameters: Retrieve all patents (max 100 at a time)
params = {
    "q": "pd within \"1782,1782\"",
    "Range": "1-100"
}

# Send request
response = requests.get(search_url, headers=headers, params=params)

# Check response
if response.status_code == 200:
    xml_data = response.text
    print("XML data received!")
else:
    print("Error:", response.text)

XML data received!


In [16]:
tree = ET.ElementTree(ET.fromstring(xml_data))
root = tree.getroot()

namespaces = {'': 'http://www.epo.org/exchange'}  # Default namespace

# Loop through each publication reference and extract the country
for publication in root.findall(".//ops:publication-reference", namespaces={"ops": "http://ops.epo.org"}):
    # Get the country from each publication reference
    country = publication.find(".//document-id/country", namespaces=namespaces)
    doc_number = publication.find(".//document-id/doc-number", namespaces=namespaces)
    
    if country is not None:
        print(country.text)
    
    if doc_number is not None:
        print(doc_number.text)
    

GB
178201321
